In [ ]:
%config Completer.use_jedi = False
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from pandas.api.types import CategoricalDtype
import math
import matplotlib as mpl
#mpl.use('pdf')
import matplotlib.pyplot as plt
import matplotlib.font_manager
import numpy as np
from collections import OrderedDict
import matplotlib
import seaborn as sns
from matplotlib.cbook import boxplot_stats
#Red
red1= sns.color_palette("Reds_d",1)
red2= sns.color_palette("Reds_d",2)
red3= sns.color_palette("Reds_d",3)
red4= sns.color_palette("Reds_d",4)
red5= sns.color_palette("Reds_d",5)
red6= sns.color_palette("Reds_d",6)
red7= sns.color_palette("Reds_d",7)
red8= sns.color_palette("Reds_d",8)

#YellowGreen
yg1= sns.color_palette("YlGn_d",1)
yg2= sns.color_palette("YlGn_d",2)
yg3= sns.color_palette("YlGn_d",3)
yg4= sns.color_palette("YlGn_d",4)
yg5= sns.color_palette("YlGn_d",5)
yg7= sns.color_palette("YlGn_d",7)

#GreenBlue
gb1= sns.color_palette("GnBu_d",1)
gb2= sns.color_palette("GnBu_d",2)
gb3= sns.color_palette("GnBu_d",3)
gb4= sns.color_palette("GnBu_d",4)
gb5= sns.color_palette("GnBu_d",5)
gb6= sns.color_palette("GnBu_d",6)
gb7= sns.color_palette("GnBu_d",7)
gb8= sns.color_palette("GnBu_d",8)

cp2 = list(map(lambda x: sns.desaturate(x,0.9),[red7[2],gb7[4]]))
cp2v1 = list(map(lambda x: sns.desaturate(x,0.9),[red7[2],yg7[0]]))
cp3 = list(map(lambda x: sns.desaturate(x,0.9),[yg7[0],gb7[4],red7[2]]))
#cp4 = list(map(lambda x: sns.desaturate(x,0.9),red1+gb2+yg1))
cp2_2 = list(map(lambda x: sns.desaturate(x,0.9),[red7[0],red7[3],gb7[4],gb7[6]]))
cp_total_spectrum = list(map(lambda x: sns.desaturate(x,0.9),gb7 + yg7 + red7))

#color_mine = colors(0)
#color_cublasxt = colors(2)
#color_ideal = colors(3)
#color_werk = colors1(2)

from pylab import cm
colors = cm.get_cmap('PuRd',  5)
viridis = cm.get_cmap('viridis',  4)
magma = cm.get_cmap('magma',  8)
cp6 = list(map(lambda x: sns.desaturate(x,0.9),[colors(2), red7[2], magma(6),  viridis(3), yg7[0], gb7[4]]))

colors_dark = cm.get_cmap('Dark2',  5)
cp5 = list(map(lambda x: sns.desaturate(x,0.9),[colors_dark(0), colors_dark(1), colors_dark(2), colors_dark(3), colors_dark(4)]))


In [ ]:
sns.set_palette(cp2)
sns.color_palette()

sns.set_palette(cp2v1)
sns.color_palette()

sns.set_palette(cp3)
sns.color_palette()

cpbasic = list(map(lambda x: sns.desaturate(x,0.9),['g','r','b','orange','grey']))
sns.set_palette(cpbasic)
sns.color_palette()

font=10
plt.rc('font', family='serif', serif='Times')
# plt.rc('text', usetex=True)
plt.rc('figure', figsize=(12,8))
plt.rc('xtick', labelsize=font-1)
plt.rc('ytick', labelsize=font-1)
plt.rc('axes', labelsize=font)
plt.rcParams.update({'axes.titlesize': font+2})

#fig, axs = plt.subplots()
#fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

#import seaborn as sns
#sns.set(style="whitegrid", palette=cpbasic,font_scale=0.2, rc={"lines.linewidth": 1})
def harry_plotter_scatter(plot_df, plot_x_axis_list, plot_y_axis_list, select_str, plot_id, 
                      rotation = 30, adj_left = .1, adj_bottom = .2, adj_right = .99, adj_top = .88, no_sort=False, hue_hue = 'None', hue_hue_order = []):
    for plot_x_itter in plot_x_axis_list:
        if no_sort:
            plot_df_sorted = plot_df
        else:
            dataTypeX = plot_df[plot_x_itter].dtype
            if dataTypeX == np.float64 or dataTypeX == np.int64 or dataTypeX == float or dataTypeX == int:
                plot_df_sorted = plot_df.sort_values(plot_x_itter)
            elif plot_x_itter == 'mem_range':
                if("FPGA" in plot_id):
                    cat_mem_range = CategoricalDtype(["[4-8]", "[8-16]", "[16-32]", "[32-64]", "[64-128]", "[128-256]", "[256-512]", "[512-1024]", "[1024-2048]"], ordered=True)
                    plot_df_sorted = plot_df
                    pd.options.mode.chained_assignment = None # https://stackoverflow.com/a/49729413
                    plot_df_sorted["mem_range"] = plot_df_sorted["mem_range"].astype(cat_mem_range)
                    plot_df_sorted = plot_df_sorted.sort_values('mem_range')                
                else:
                    plot_df_sorted = plot_df.sort_values('A_mem_footprint')
            else:
                print("Warning: X not sorted: dtype = %s" % (dataTypeX))
                plot_df_sorted = plot_df
        for plot_y_itter in plot_y_axis_list:
            fig, axs = plt.subplots()
            if hue_hue == 'None':
                sns_plot = sns.scatterplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, ax=axs)
            else:
                sns_plot = sns.scatterplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, hue=hue_hue, hue_order=hue_hue_order, ax=axs)
                
            if(hue_hue_order != []):               
                # reordering the labels
                handles, labels = plt.gca().get_legend_handles_labels()
                order = [labels.index(i) for i in hue_hue_order]
                plt.legend([handles[i] for i in order], [labels[i] for i in order], title = hue_hue.replace("_categ",""), fancybox=True)

            axs.set_title('Select: %s' %(select_str))
            fig.subplots_adjust(left=adj_left, bottom=adj_bottom, right=adj_right, top=adj_top)
            plt.xticks(rotation=rotation)
            select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
            fig.savefig('./Paper_plots/%s_Select-%s_x-%s_y-%s.pdf' % (plot_id, select_str_id, plot_x_itter, plot_y_itter))
            plt.close()
            
def harry_plotter_box(plot_df, plot_x_axis_list, plot_y_axis_list, select_str, plot_id, 
                      rotation = 30, adj_left = .1, adj_bottom = .2, adj_right = .99, adj_top = .88, no_sort=False, hue_hue = 'None', hue_hue_order = []):
    for plot_x_itter in plot_x_axis_list:
        if no_sort:
            plot_df_sorted = plot_df
        else:
            dataTypeX = plot_df[plot_x_itter].dtype
            if dataTypeX == np.float64 or dataTypeX == np.int64 or dataTypeX == float or dataTypeX == int:
                plot_df_sorted = plot_df.sort_values(plot_x_itter)
            elif plot_x_itter == 'mem_range':
                if("FPGA" in plot_id):
                    cat_mem_range = CategoricalDtype(["[4-8]", "[8-16]", "[16-32]", "[32-64]", "[64-128]", "[128-256]", "[256-512]", "[512-1024]", "[1024-2048]"], ordered=True)
                    plot_df_sorted = plot_df
                    pd.options.mode.chained_assignment = None # https://stackoverflow.com/a/49729413
                    plot_df_sorted["mem_range"] = plot_df_sorted["mem_range"].astype(cat_mem_range)
                    plot_df_sorted = plot_df_sorted.sort_values('mem_range')                
                else:
                    plot_df_sorted = plot_df.sort_values('A_mem_footprint')
            else:
                print("Warning: X not sorted: dtype = %s" % (dataTypeX))
                plot_df_sorted = plot_df
        for plot_y_itter in plot_y_axis_list:
            fig, axs = plt.subplots()
            if hue_hue == 'None':
                sns_plot = sns.boxplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, ax=axs)
            else:
                if(hue_hue_order != []):
                    sns_plot = sns.boxplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, hue=hue_hue, hue_order=hue_hue_order, ax=axs)
                else:
                    sns_plot = sns.boxplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, hue=hue_hue, ax=axs)

            if(hue_hue_order != []):               
                # reordering the labels
                handles, labels = plt.gca().get_legend_handles_labels()
                order = [labels.index(i) for i in hue_hue_order]
                plt.legend([handles[i] for i in order], [labels[i] for i in order], title = hue_hue.replace("_categ",""), fancybox=True)

            axs.set_title('Select: %s' %(select_str))
            fig.subplots_adjust(left=adj_left, bottom=adj_bottom, right=adj_right, top=adj_top)
            plt.xticks(rotation=rotation)
            select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
            fig.savefig('./Paper_plots/%s_Select-%s_x-%s_y-%s.pdf' % (plot_id, select_str_id, plot_x_itter, plot_y_itter))
            plt.close()

            
def harry_plotter_violinplot(plot_df, plot_x_axis_list, plot_y_axis_list, select_str, plot_id, 
                      rotation = 30, adj_left = .1, adj_bottom = .2, adj_right = .99, adj_top = .88, no_sort=False, hue_hue = 'None', hue_hue_order = []):
    for plot_x_itter in plot_x_axis_list:
        if no_sort:
            plot_df_sorted = plot_df
        else:
            dataTypeX = plot_df[plot_x_itter].dtype
            if dataTypeX == np.float64 or dataTypeX == np.int64 or dataTypeX == float or dataTypeX == int:
                plot_df_sorted = plot_df.sort_values(plot_x_itter)
            elif plot_x_itter == 'mem_range':
                if("FPGA" in plot_id):
                    cat_mem_range = CategoricalDtype(["[4-8]", "[8-16]", "[16-32]", "[32-64]", "[64-128]", "[128-256]", "[256-512]", "[512-1024]", "[1024-2048]"], ordered=True)
                    plot_df_sorted = plot_df
                    pd.options.mode.chained_assignment = None # https://stackoverflow.com/a/49729413
                    plot_df_sorted["mem_range"] = plot_df_sorted["mem_range"].astype(cat_mem_range)
                    plot_df_sorted = plot_df_sorted.sort_values('mem_range')                
                else:
                    plot_df_sorted = plot_df.sort_values('A_mem_footprint')
            else:
                print("Warning: X not sorted: dtype = %s" % (dataTypeX))
                plot_df_sorted = plot_df
        for plot_y_itter in plot_y_axis_list:
            fig, axs = plt.subplots()
            if hue_hue == 'None':
                sns_plot = sns.violiplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, ax=axs)
            else:
                if(hue_hue_order != []):
                    sns_plot = sns.violinplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, hue=hue_hue, hue_order=hue_hue_order, ax=axs)
                else:
                    sns_plot = sns.violinplot(data=plot_df_sorted, x=plot_x_itter, y=plot_y_itter, hue=hue_hue, ax=axs)

            if(hue_hue_order != []):               
                # reordering the labels
                handles, labels = plt.gca().get_legend_handles_labels()
                order = [labels.index(i) for i in hue_hue_order]
                plt.legend([handles[i] for i in order], [labels[i] for i in order], title = hue_hue.replace("_categ",""), fancybox=True)

            axs.set_title('Select: %s' %(select_str))
            fig.subplots_adjust(left=adj_left, bottom=adj_bottom, right=adj_right, top=adj_top)
            plt.xticks(rotation=rotation)
            select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
            fig.savefig('./Paper_plots/%s_Select-%s_x-%s_y-%s.pdf' % (plot_id, select_str_id, plot_x_itter, plot_y_itter))
            plt.close()


# Read GPU Data (panastas)

In [ ]:
header_names = ['mtx_name','distribution','placement','seed',
                'm','n','nz','density','A_mem_footprint','mem_range',
                'avg_nz_row','std_nz_row',
                'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                'skew_coeff','avg_num_neighbours','cross_row_similarity',
                'implementation','time','gflops','W_avg','J_estimated', 'System', 'Arch','friends']

# select for CPU which data to read (used later in validation too)
Hawk_threads = 64
#Hawk_threads = 128
Arm_threads = 80
# Arm_threads = 160

In [ ]:
dtypes=['D']
for dtype in dtypes:

    inputdata_GPU_V100_friends = pd.read_csv('../benchmark_results/vulcan-V100/vulcan-V100_dtype-%s_run_friend_dataset.csv' % dtype, names = header_names)
    inputdata_GPU_V100_friends['System'] = 'Tesla-V100'
    
    inputdata_GPU_P100_friends = pd.read_csv('../benchmark_results/vulcan-P100/vulcan-P100_dtype-%s_run_friend_dataset.csv' % dtype, names = header_names)
    inputdata_GPU_P100_friends['System'] = 'Tesla-P100'
    
    # inputdata_GPU_A100_friends = pd.read_csv('../benchmark_results/epyc5A100/epyc5A100_dtype-%s_run_friend_dataset.csv' % dtype, names = header_names)
    # inputdata_GPU_A100_friends['System'] = 'Tesla-A100'
    
    # inputdata_GPU = pd.concat([inputdata_GPU_V100_friends, inputdata_GPU_P100_friends, inputdata_GPU_A100_friends])
    inputdata_GPU = pd.concat([inputdata_GPU_V100_friends, inputdata_GPU_P100_friends])
    inputdata_GPU['friends'] = True
    inputdata_GPU['Arch'] = 'GPU'

    print(inputdata_GPU.shape)
    # print(inputdata_GPU.head(2))
    #inputdata_GPU = inputdata_GPU[inputdata_GPU.implementation != 'CSR5_9']
    #inputdata_GPU = inputdata_GPU[inputdata_GPU.implementation != 'cuSPARSE_hyb9-2']
    print(set(inputdata_GPU['implementation']))

# Read CPU Data (dgal)

In [ ]:
    inputdata_CPU_AMD_friends = pd.read_csv('../benchmark_results/amd-epyc/amd-epyc_friends_10_samples_30_range_t%d_d.csv' % Hawk_threads, names=header_names)
    inputdata_CPU_AMD_friends['System'] = 'Amd-Epyc'
    

    inputdata_CPU_ARM_friends = pd.read_csv('../benchmark_results/arm/arm_friends_10_samples_30_range_t%d_d.csv' % Arm_threads, names=header_names)
    inputdata_CPU_ARM_friends['System'] = 'Arm-Neon'
    
    inputdata_CPU = pd.concat([inputdata_CPU_AMD_friends, inputdata_CPU_ARM_friends])
    inputdata_CPU['friends'] = True
    inputdata_CPU['Arch'] = 'CPU'    
    
    print(inputdata_CPU.shape)
    # print(inputdata_CPU.head(2))
    print(set(inputdata_CPU['implementation']))

# Read FPGA Data (pmpakos)

In [ ]:
    fname = "alveo-u280_spmv_validation_matrices_10_samples_30_range_twins_dtype-%s.csv" % dtype
    inputdata_FPGA_friends = pd.read_csv('../benchmark_results/alveo-u280/%s' % fname, names = header_names)
    inputdata_FPGA_friends['System'] = 'Alveo-U280'

    inputdata_FPGA = inputdata_FPGA_friends
    inputdata_FPGA['friends'] = True
    inputdata_FPGA['Arch'] = 'FPGA'

    print(inputdata_FPGA.shape)
    # print(inputdata_FPGA.head(2))
    print(set(inputdata_FPGA['implementation']))

# Concatenate all data, place in *inputdata* dataframe

In [ ]:
    # Merge the results
    inputdata = pd.concat([inputdata_GPU,inputdata_CPU,inputdata_FPGA])
    print(inputdata.shape)

    # Group per reps, take mean
    groupreps = inputdata.groupby(['mtx_name','distribution','placement','seed',
                                   'm','n','nz','density','A_mem_footprint','mem_range',
                                   'avg_nz_row','std_nz_row',
                                   'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                                   'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                                   'skew_coeff','avg_num_neighbours','cross_row_similarity',
                                   'implementation','System', 'Arch','friends']).mean().reset_index().reindex(columns=header_names)

    # dataTypeSeries = groupreps.dtypes
    # print('Data type of each column of Dataframe :')
    # print(dataTypeSeries)
    print(groupreps.shape)
    group_system_best = groupreps

---
# Group by "best-of" implementation for each device
# skip this step if you want to plot every measurement collected

In [ ]:
    # Group per system, take best (it was over "inputdata", but "groupreps" is better choice I think)
    # fixed this after reordering groupreps columns according to header_names
    # before this, columns and data were mixed and it was a complete shitstorm
    group_system = groupreps.groupby(['mtx_name','distribution','placement','seed',
                                      'm','n','nz','density','A_mem_footprint','mem_range',
                                      'avg_nz_row','std_nz_row',
                                      'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                                      'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                                      'skew_coeff','avg_num_neighbours','cross_row_similarity',
                                      'System','Arch','friends'], as_index = False)
    reslist = []
    for desc, experiment in group_system:
        best_format = experiment['implementation'].iloc[experiment['gflops'].argmax()]
        outrow = experiment[experiment['implementation'] == best_format]
        # if(len(outrow)>1):
        #     print(len(outrow), outrow)
        reslist.append(outrow.values.tolist()[0])
        # print(outrow.values.tolist()[0])
    group_system_best = pd.DataFrame(reslist, columns = header_names)

    print(group_system_best.shape)

---
---
---
# Validation matrices 

In [ ]:
header_names = ['mtx_name','distribution','placement','seed',
                'm','n','nz','density','A_mem_footprint','mem_range',
                'avg_nz_row','std_nz_row',
                'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                'skew_coeff', 'avg_num_neighbours','cross_row_similarity',
                'implementation','time','gflops','W_avg','J_estimated', 'System', 'Arch']

def find_class(mem_footprint):
    low_mb_list =   [4,8, 16,32,64, 128,256,512, 1024,2048, 4096] # removed largest mem range (2048-4096)
    high_mb_list =  [8,16,32,64,128,256,512,1024,2048,4096, 8192]
    for i in range(len(low_mb_list)):
        if(mem_footprint>=low_mb_list[i] and mem_footprint<=high_mb_list[i]):
            pos = i
            mem_range = '['+str(low_mb_list[pos])+'-'+str(high_mb_list[pos])+']'
            return mem_range
    return str(-1)

---
# Read GPU Data (panastas)

In [ ]:
dtypes=['D']
for dtype in dtypes:
    def modify_footprint(v):
        if(v["implementation"]=="CSR5_9"):
            return (v["A_mem_footprint"] - 4*(v["m"]+v["n"]))/(1024*1024.0)
        else:
            return v["A_mem_footprint"]

    def create_complete_gpu_csv(gpu_dataframe, system, arch):
        gpu_dataframe["mtx_name"] = gpu_dataframe["mtx_name"].apply(lambda x: x.split("/")[-1].split(".")[0])
        vm_features = pd.read_csv("../benchmark_results/validation_matrices_features.csv", sep="\t")
        mtx_names = list(vm_features["matrix"]) # same as : list(set(inputdata_GPU["mtx_name"]))

        inputvaldata_GPU = pd.DataFrame(columns=header_names)
        
        for mtx_name in mtx_names:
            for index, curr in gpu_dataframe[gpu_dataframe["mtx_name"] == mtx_name].iterrows():
                mtx_name = mtx_name
                distribution = "unused"
                placement = "unused"
                seed = curr["seed"]
                m = curr["m"]
                n = curr["n"]
                nz = curr["nz"]
                density = nz/(m*n)*100.0
                A_mem_footprint = curr["A_mem_footprint"]
                mem_range = find_class((nz*(64+32)+32*(m+1))/(8*1024*1024))
                avg_nz_row = list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-avg"])[0]
                std_nz_row = list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-std"])[0]
                avg_bandwidth = list(vm_features[vm_features["matrix"]==mtx_name]["bw-avg"])[0]*n
                std_bandwidth = list(vm_features[vm_features["matrix"]==mtx_name]["bw-std"])[0]*n
                avg_bandwidth_scaled = avg_bandwidth/n
                std_bandwidth_scaled = std_bandwidth/n
                # avg_scattering = list(vm_features[vm_features["matrix"]==mtx_name]["sc-avg"])[0]
                # std_scattering = list(vm_features[vm_features["matrix"]==mtx_name]["sc-std"])[0]
                # avg_scattering_scaled = avg_scattering*n
                # std_scattering_scaled = std_scattering*n
                # skew_coeff = (list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-max"])[0] - avg_nz_row)/avg_nz_row
                skew_coeff = list(vm_features[vm_features["matrix"]==mtx_name]["skew_coeff"])[0]
                avg_num_neighbours = list(vm_features[vm_features["matrix"]==mtx_name]["neigh-avg"])[0]
                cross_row_similarity = list(vm_features[vm_features["matrix"]==mtx_name]["cross_row_sim-avg"])[0]
                
                implementation = curr["implementation"]
                time = curr["time"]
                gflops = curr["gflops"]
                W_avg = curr["W_avg"]
                J_estimated = curr["J_estimated"]
                inputvaldata_GPU = inputvaldata_GPU.append({'mtx_name' : mtx_name, 'distribution' : distribution, 'placement' : placement, 'seed' : seed, 
                                                            'm' : m, 'n' : n, 'nz' : nz, 'density' : density, 'A_mem_footprint' : A_mem_footprint, 'mem_range' : mem_range, 
                                                            'avg_nz_row' : avg_nz_row, 'std_nz_row' : std_nz_row, 
                                                            'avg_bandwidth' : avg_bandwidth, 'std_bandwidth' : std_bandwidth, 'avg_bandwidth_scaled' : avg_bandwidth_scaled, 'std_bandwidth_scaled' : std_bandwidth_scaled,
                                                            'skew_coeff' : skew_coeff, 'avg_num_neighbours' : avg_num_neighbours, 'cross_row_similarity' : cross_row_similarity,
                                                            'implementation' : implementation, 'time' : time, 'gflops' : gflops, 'W_avg' : W_avg, 'J_estimated' : J_estimated},
                                                           ignore_index=True)
        inputvaldata_GPU['System'] = system
        inputvaldata_GPU['Arch'] = arch
        if(system=='Tesla-V100'):
            inputvaldata_GPU["A_mem_footprint"] = inputvaldata_GPU.apply(lambda x: modify_footprint(x), axis=1)
        return inputvaldata_GPU

# read V100 first

In [ ]:
    arch, system = 'GPU', 'Tesla-V100'
    
    # fname = "silver1-TeslaV100_dtype-%s_run_validation_matrices.csv" % dtype
    fname = "vulcan-V100/vulcan-V100_dtype-%s_run_validation_matrices.csv" % dtype
    gpu_data = pd.read_csv('../benchmark_results/%s' % fname, names = ['mtx_name','distribution','placement','diagonal_factor','seed',
                                                                'm','n','nz','density','A_mem_footprint','mem_range',
                                                                'avg_nz_row','std_nz_row',
                                                                'avg_bandwidth','std_bandwidth','avg_scattering','std_scattering',
                                                                'implementation','time','gflops','W_avg', 'J_estimated'])

    gpu_data = create_complete_gpu_csv(gpu_data, system, arch)
    fname2 = fname.replace('.csv', '_modified_features.csv')
    gpu_data.to_csv('../benchmark_results/%s' % fname2, header=False, index=False)
    print(gpu_data.head(2))

# read P100 data

In [ ]:
    arch, system = 'GPU', 'Tesla-P100'
    
    fname = "vulcan-P100/vulcan-P100_dtype-%s_run_validation_matrices.csv" % dtype
    # it has different data layout @@@ panastas @@@
    gpu_data = pd.read_csv('../benchmark_results/%s' % fname, names = ['mtx_name','distribution','placement','seed',
                                                                'm','n','nz','density','A_mem_footprint','mem_range',
                                                                'avg_nz_row','std_nz_row',
                                                                'avg_bandwidth','std_bandwidth','avg_scattering','std_scattering',
                                                                '1','2','3','4','5','6','7',
                                                                'implementation','time','gflops','W_avg', 'J_estimated'])
    
    gpu_data = create_complete_gpu_csv(gpu_data, system, arch)
    fname2 = fname.replace('.csv', '_modified_features.csv')
    gpu_data.to_csv('../benchmark_results/%s' % fname2, header=False, index=False)
    print(gpu_data.head(2))

---
# Read A100 data

In [ ]:
    # arch, system = 'GPU', 'Tesla-A100'

    # fname = "epyc5A100/epyc5A100_dtype-%s_run_validation_matrices.csv" % dtype
    # # it has different data layout @@@ panastas @@@
    # gpu_data = pd.read_csv('../benchmark_results/%s' % fname, names = ['mtx_name','distribution','placement','seed',
    #                                                             'm','n','nz','density','A_mem_footprint','mem_range',
    #                                                             'avg_nz_row','std_nz_row',
    #                                                             'avg_bandwidth','std_bandwidth','avg_scattering','std_scattering',
    #                                                             '1','2','3','4','5','6','7',
    #                                                             'implementation','time','gflops','W_avg', 'J_estimated'])

    # gpu_data = create_complete_gpu_csv(gpu_data, system, arch)
    # fname2 = fname.replace('.csv', '_modified_features.csv')
    # gpu_data.to_csv('../benchmark_results/%s' % fname2, header=False, index=False)
    # print(gpu_data.head(2))

---
# Read CPU Data (dgal)

In [ ]:
    def create_complete_cpu_csv(cpu_dataframe, system, arch):
        vm_features = pd.read_csv("../benchmark_results/validation_matrices_features.csv", sep="\t")
        mtx_names = list(vm_features["matrix"]) # same as : list(set(inputdata_GPU["mtx_name"]))

        inputvaldata_CPU = pd.DataFrame(columns=header_names)
        
        for mtx_name in mtx_names:
            for index, curr in cpu_dataframe[cpu_dataframe["mtx_name"] == mtx_name].iterrows():
                mtx_name = mtx_name
                distribution = "unused"
                placement = "unused"
                diagonal_factor = 0
                seed = 0
                m = curr["m"]
                n = curr["n"]
                nz = curr["nz"]
                density = nz/(m*n)*100.0
                A_mem_footprint = curr["A_mem_footprint"]
                mem_range = find_class((nz*(64+32)+32*(m+1))/(8*1024*1024))
                avg_nz_row = list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-avg"])[0]
                std_nz_row = list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-std"])[0]
                avg_bandwidth = list(vm_features[vm_features["matrix"]==mtx_name]["bw-avg"])[0]*n
                std_bandwidth = list(vm_features[vm_features["matrix"]==mtx_name]["bw-std"])[0]*n
                avg_bandwidth_scaled = avg_bandwidth/n
                std_bandwidth_scaled = std_bandwidth/n
                # avg_scattering = list(vm_features[vm_features["matrix"]==mtx_name]["sc-avg"])[0]
                # std_scattering = list(vm_features[vm_features["matrix"]==mtx_name]["sc-std"])[0]
                # avg_scattering_scaled = avg_scattering*n
                # std_scattering_scaled = std_scattering*n
                # skew_coeff = (list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-max"])[0] - avg_nz_row)/avg_nz_row
                skew_coeff = list(vm_features[vm_features["matrix"]==mtx_name]["skew_coeff"])[0]
                avg_num_neighbours = list(vm_features[vm_features["matrix"]==mtx_name]["neigh-avg"])[0]
                cross_row_similarity = list(vm_features[vm_features["matrix"]==mtx_name]["cross_row_sim-avg"])[0]
                                
                implementation = curr["implementation"]
                time = curr["time"]
                gflops = curr["gflops"]
                W_avg = curr["W_avg"]
                J_estimated = curr["J_estimated"]
                inputvaldata_CPU = inputvaldata_CPU.append({'mtx_name' : mtx_name, 'distribution' : distribution, 'placement' : placement, 'seed' : seed, 
                                                              'm' : m, 'n' : n, 'nz' : nz, 'density' : density, 'A_mem_footprint' : A_mem_footprint, 'mem_range' : mem_range, 
                                                              'avg_nz_row' : avg_nz_row, 'std_nz_row' : std_nz_row, 
                                                              'avg_bandwidth' : avg_bandwidth, 'std_bandwidth' : std_bandwidth, 'avg_bandwidth_scaled' : avg_bandwidth_scaled, 'std_bandwidth_scaled' : std_bandwidth_scaled,
                                                              #'avg_scattering' : avg_scattering, 'std_scattering' : std_scattering, 'avg_scattering_scaled' : avg_scattering_scaled, 'std_scattering_scaled' : std_scattering_scaled,
                                                              'skew_coeff' : skew_coeff, 'avg_num_neighbours' : avg_num_neighbours, 'cross_row_similarity' : cross_row_similarity,
                                                              'implementation' : implementation, 'time' : time, 'gflops' : gflops, 'W_avg' : W_avg, 'J_estimated' : J_estimated},
                                                             ignore_index=True)
        inputvaldata_CPU['System'] = system
        inputvaldata_CPU['Arch'] = arch
        return inputvaldata_CPU

# read Hawk data first

In [ ]:
    arch, system = 'CPU', 'Amd-Epyc'
    fname = "amd-epyc/amd-epyc_validation_matrices_t%d_d.csv" % Hawk_threads

    cpu_data = pd.read_csv('../benchmark_results/%s' % fname, names = ['mtx_name','distribution','placement','seed',
                'm','n','nz','density','A_mem_footprint','mem_range',
                'avg_nz_row','std_nz_row',
                'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                'skew_coeff',
                'implementation','time','gflops','W_avg','J_estimated', 'System'])
      
    cpu_data = create_complete_cpu_csv(cpu_data, system, arch)
    fname2 = fname.replace('.csv', '_modified_features.csv')
    
    if(fname == "amd-epyc/amd-epyc_validation_matrices_t128_d.csv"):
        cpu_data = cpu_data[cpu_data['implementation'] != 'MKL_IE_no_optimize']
        cpu_data = cpu_data[cpu_data['implementation'] != 'Custom_CSR_BV_CPU']
        cpu_data = cpu_data[cpu_data['implementation'] != 'Custom_CSR_B_CPU']
        # cpu_data = cpu_data[cpu_data['implementation'] != 'MKL_IE_no_hint']
        # cpu_data = cpu_data[cpu_data['implementation'] != 'MKL_IE']
    
    cpu_data.to_csv('../benchmark_results/%s' % fname2, header=False, index=False)
    print(cpu_data.head(2))
    print(set(cpu_data['implementation']))

# read Arm data

In [ ]:
    arch, system = 'CPU', 'Arm-Neon'
    fname = "arm/arm_validation_matrices_t%d_d.csv" % Arm_threads

    cpu_data_arm = pd.read_csv('../benchmark_results/%s' % fname, names = ['mtx_name','distribution','placement','seed',
                'm','n','nz','density','A_mem_footprint','mem_range',
                'avg_nz_row','std_nz_row',
                'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                'avg_scattering','std_scattering','avg_scattering_scaled','std_scattering_scaled',
                'skew_coeff',
                'implementation','time','gflops','W_avg','J_estimated', 'System'])

    cpu_data_arm = create_complete_cpu_csv(cpu_data_arm, system, arch)
    fname2 = fname.replace('.csv', '_modified_features.csv')
    
    cpu_data_arm.to_csv('../benchmark_results/%s' % fname2, header=False, index=False)
    print(cpu_data_arm.head(2))
    print(set(cpu_data_arm['implementation']))

---
# Read FPGA Data (pmpakos)

In [ ]:
def create_complete_fpga_csv(fpga_dataframe, system, arch):
        vm_features = pd.read_csv("../benchmark_results/validation_matrices_features.csv", sep="\t")
        mtx_names = list(vm_features["matrix"]) # same as : list(set(inputdata_GPU["mtx_name"]))

        inputvaldata_FPGA = pd.DataFrame(columns=header_names)
        
        for mtx_name in mtx_names:
            pin_df = fpga_dataframe[fpga_dataframe["matrix"] == mtx_name]
            if(pin_df.empty):
                m = list(vm_features[vm_features["matrix"]==mtx_name]["nr_rows"])[0]
                n = list(vm_features[vm_features["matrix"]==mtx_name]["nr_cols"])[0]
                nz = list(vm_features[vm_features["matrix"]==mtx_name]["nr_nnzs"])[0]
                A_mem_footprint = (nz*(64+32)+32*(m+1))/(8*1024*1024)
                inputvaldata_FPGA = inputvaldata_FPGA.append({'mtx_name' : mtx_name, 'distribution' : "unused", 'placement' : "unused", 'seed' : 0, 'm' : m, 'n' : n, 'nz' : nz, 'density' : nz/(m*n)*100.0, 'A_mem_footprint' : A_mem_footprint, 'mem_range' : find_class((nz*(64+32)+32*(m+1))/(8*1024*1024)),'avg_nz_row' : list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-avg"])[0],'std_nz_row' : list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-std"])[0], 'avg_bandwidth' : list(vm_features[vm_features["matrix"]==mtx_name]["bw-avg"])[0]*n,'std_bandwidth' : list(vm_features[vm_features["matrix"]==mtx_name]["bw-std"])[0]*n,'avg_bandwidth_scaled' : list(vm_features[vm_features["matrix"]==mtx_name]["bw-avg"])[0],'std_bandwidth_scaled' : list(vm_features[vm_features["matrix"]==mtx_name]["bw-std"])[0],'skew_coeff' : list(vm_features[vm_features["matrix"]==mtx_name]["skew_coeff"])[0],'avg_num_neighbours' : list(vm_features[vm_features["matrix"]==mtx_name]["neigh-avg"])[0],'cross_row_similarity' : list(vm_features[vm_features["matrix"]==mtx_name]["cross_row_sim-avg"])[0],'implementation' : 'unused','time' : -1,'gflops' : -1,'W_avg' : -1,'J_estimated' : -1},ignore_index=True)
            for index, curr in pin_df.iterrows():
                mtx_name = mtx_name
                distribution = "unused"
                placement = "unused"
                diagonal_factor = 0
                seed = 0
                m = curr["nr_rows"]
                n = curr["nr_cols"]
                nz = curr["nr_nnz"]
                density = nz/(m*n)*100.0
                # A_mem_footprint = curr["mem_footprint"]
                A_mem_footprint = (nz*(64+32)+32*(m+1))/(8*1024*1024)

                mem_range = find_class((nz*(64+32)+32*(m+1))/(8*1024*1024))
                avg_nz_row = list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-avg"])[0]
                std_nz_row = list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-std"])[0]
                avg_bandwidth = list(vm_features[vm_features["matrix"]==mtx_name]["bw-avg"])[0]*n
                std_bandwidth = list(vm_features[vm_features["matrix"]==mtx_name]["bw-std"])[0]*n
                avg_bandwidth_scaled = avg_bandwidth/n
                std_bandwidth_scaled = std_bandwidth/n
                # avg_scattering = list(vm_features[vm_features["matrix"]==mtx_name]["sc-avg"])[0]
                # std_scattering = list(vm_features[vm_features["matrix"]==mtx_name]["sc-std"])[0]
                # avg_scattering_scaled = avg_scattering*n
                # std_scattering_scaled = std_scattering*n
                # skew_coeff = (list(vm_features[vm_features["matrix"]==mtx_name]["nnz-r-max"])[0] - avg_nz_row)/avg_nz_row
                skew_coeff = list(vm_features[vm_features["matrix"]==mtx_name]["skew_coeff"])[0]
                avg_num_neighbours = list(vm_features[vm_features["matrix"]==mtx_name]["neigh-avg"])[0]
                cross_row_similarity = list(vm_features[vm_features["matrix"]==mtx_name]["cross_row_sim-avg"])[0]

                implementation = curr["implementation"]
                time = curr["runtime_iter"]
                gflops = curr["gflops"]
                W_avg = curr["W_avg"]
                J_estimated = curr["J_estimated"]
                inputvaldata_FPGA = inputvaldata_FPGA.append({'mtx_name' : mtx_name, 'distribution' : distribution, 'placement' : placement, 'seed' : seed, 
                                                              'm' : m, 'n' : n, 'nz' : nz, 'density' : density, 'A_mem_footprint' : A_mem_footprint, 'mem_range' : mem_range, 
                                                              'avg_nz_row' : avg_nz_row, 'std_nz_row' : std_nz_row, 
                                                              'avg_bandwidth' : avg_bandwidth, 'std_bandwidth' : std_bandwidth, 'avg_bandwidth_scaled' : avg_bandwidth_scaled, 'std_bandwidth_scaled' : std_bandwidth_scaled,
                                                              #'avg_scattering' : avg_scattering, 'std_scattering' : std_scattering, 'avg_scattering_scaled' : avg_scattering_scaled, 'std_scattering_scaled' : std_scattering_scaled,
                                                              'skew_coeff' : skew_coeff, 'avg_num_neighbours' : avg_num_neighbours, 'cross_row_similarity' : cross_row_similarity,
                                                              'implementation' : implementation, 'time' : time, 'gflops' : gflops, 'W_avg' : W_avg, 'J_estimated' : J_estimated},
                                                             ignore_index=True)
        inputvaldata_FPGA['System'] = system
        inputvaldata_FPGA['Arch'] = arch
        return inputvaldata_FPGA

In [ ]:
    arch, system = 'FPGA', 'Alveo-U280'

    fname = "results_validation_matrices.csv"
    fpga_data = pd.read_csv('../benchmark_code/FPGA/result_parsing/%s' % fname, names = ["matrix","nr_rows","nr_cols","nr_nnz","density",
                                                                 "mem_footprint","implementation","runtime_iter",
                                                                 "gflops","W_avg","J_estimated"])
    
    fpga_data = create_complete_fpga_csv(fpga_data, system, arch)
    fname2 = "alveo-u280/alveo-u280_spmv_validation_matrices_dtype-D.csv"
    fpga_data.to_csv('../benchmark_results/%s' % fname2, header=False, index=False)
    print(fpga_data.head(2))

---
# Ready to read validation matrices complete dataset!

In [ ]:
dtypes=['D']

for dtype in dtypes:
    #Unified read
    # fname = "silver1-TeslaV100_dtype-%s_run_validation_matrices_modified_features.csv" % dtype
    fname = "vulcan-V100/vulcan-V100_dtype-%s_run_validation_matrices_modified_features.csv" % dtype
    inputvaldata_GPU_V100 = pd.read_csv('../benchmark_results/%s' % fname, names = header_names,index_col=False)
    
    # fname = "silver1-TeslaV100_dtype-%s_run_validation_matrices_modified_features.csv" % dtype
    fname = "vulcan-P100/vulcan-P100_dtype-%s_run_validation_matrices_modified_features.csv" % dtype
    inputvaldata_GPU_P100 = pd.read_csv('../benchmark_results/%s' % fname, names = header_names,index_col=False)
    
    # # fname = "epyc5TeslaA100_dtype-%s_run_validation_matrices_modified_features.csv" % dtype
    # fname = "epyc5A100/epyc5A100_dtype-%s_run_validation_matrices_modified_features.csv" % dtype
    # inputvaldata_GPU_A100 = pd.read_csv('../benchmark_results/%s' % fname, names = header_names,index_col=False)
    
    #Device P100 V100 A100
    #specification 720 980.99 1555.00
    #copy 720 825.47 1329.58
    #scale 720 826.52 1327.59
    #add 720 873.63 1376.84
    #triadd 720 872.37 1377.21
  
    inputvaldata_GPU_P100['mem_bw_gbytes_s'] = 464
    inputvaldata_GPU_P100['cache_bw_gbytes_s'] = 464
    inputvaldata_GPU_V100['mem_bw_gbytes_s'] = 760
    inputvaldata_GPU_V100['cache_bw_gbytes_s'] = 760
    # inputvaldata_GPU_A100['mem_bw_gbytes_s'] = 1350
    # inputvaldata_GPU_A100['cache_bw_gbytes_s'] = 1350
    
    fname = "amd-epyc/amd-epyc_validation_matrices_t%d_d_modified_features.csv" % Hawk_threads
    inputvaldata_CPU_Hawk = pd.read_csv('../benchmark_results/%s' % fname, names = header_names,index_col=False)
    
    fname = "arm/arm_validation_matrices_t%d_d_modified_features.csv" % Arm_threads
    inputvaldata_CPU_Arm = pd.read_csv('../benchmark_results/%s' % fname, names = header_names,index_col=False)
    
    fname = "alveo-u280/alveo-u280_spmv_validation_matrices_dtype-%s.csv" % dtype
    inputvaldata_FPGA = pd.read_csv('../benchmark_results/%s' % fname, names = header_names, index_col=False)
    
    inputvaldata_CPU_Hawk['mem_bw_gbytes_s'] = 105
    inputvaldata_CPU_Hawk['cache_bw_gbytes_s'] = 878
    inputvaldata_CPU_Arm['mem_bw_gbytes_s'] = 102
    inputvaldata_CPU_Arm['cache_bw_gbytes_s'] = 650
    inputvaldata_FPGA['mem_bw_gbytes_s'] = 287.5 #  20/32 * 460 = 287.5 GB/s (
    inputvaldata_FPGA['cache_bw_gbytes_s'] = 287.5

# Merge results

In [ ]:
    # Merge the results
    inputvaldata = pd.concat([inputvaldata_GPU_V100, inputvaldata_GPU_P100, # inputvaldata_GPU_A100,
                              inputvaldata_CPU_Hawk, inputvaldata_CPU_Arm, 
                              inputvaldata_FPGA])
    print(inputvaldata.shape)
    #inputvaldata = inputvaldata[inputvaldata.implementation != 'CSR5_9']
    #inputvaldata = inputvaldata[inputvaldata.implementation != 'cuSPARSE_hyb9-2']
    
    header_names.append('mem_bw_gbytes_s')
    header_names.append('cache_bw_gbytes_s')
        
    # Group per reps, take mean
    # header_names without 'time','gflops','W_avg','J_estimated'
    groupvalreps = inputvaldata.groupby( ['mtx_name','distribution','placement','seed',
                                          'm','n','nz','density','A_mem_footprint','mem_range',
                                          'avg_nz_row','std_nz_row',
                                          'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                                          'skew_coeff','avg_num_neighbours','cross_row_similarity',
                                          'implementation','System', 'Arch', 'mem_bw_gbytes_s','cache_bw_gbytes_s']).mean().reset_index().reindex(columns=header_names)
    
    # groupvalreps = groupvalreps[groupvalreps['implementation'] != 'MKL_IE_Optimize']
    # groupvalreps = groupvalreps[groupvalreps['implementation'] != 'MKL_IE']
    print(set(groupvalreps['implementation']))
    #print(groupvalreps[groupvalreps["mtx_name"]=="circuit5M"])
    print(groupvalreps.shape)
    group_val_system_best = groupvalreps

---
# Group by "best-of" implementation for each device
# skip this step if you want to plot per-format validation plots

In [ ]:
    # Group per system, take best
    # header_names without 'time','gflops','W_avg','J_estimated', 'implementation'
    # Curiously, I have to also remove "A_mem_footprint", because for CSR5 it reports different mem_footprint
    # for the same matrix, leading to a different representation in groupby
    groupval_system = groupvalreps.groupby(['mtx_name','distribution','placement','seed',
                                            'm','n','nz','density','mem_range',
                                            'avg_nz_row','std_nz_row',
                                            'avg_bandwidth','std_bandwidth','avg_bandwidth_scaled','std_bandwidth_scaled',
                                            'skew_coeff','avg_num_neighbours','cross_row_similarity',
                                            'System','Arch', 'mem_bw_gbytes_s','cache_bw_gbytes_s'], as_index = False)
    reslist = []
    
    for desc, experiment in groupval_system:
        best_format = experiment['implementation'].iloc[experiment['gflops'].argmax()]
        outrow = experiment[experiment['implementation'] == best_format]
        outrow = outrow[header_names] # reorder column because they are mixed by group-by
        reslist.append(outrow.values.tolist()[0])

    group_val_system_best = pd.DataFrame(reslist, columns = header_names)
    print(group_val_system_best.shape)    

# Some fixes that have to be done by hand

In [ ]:
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'scircuit') , ['A_mem_footprint']] =  11.63
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'mac_econ_fwd500') , ['A_mem_footprint']] =  15.36
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'raefsky3') , ['A_mem_footprint']] =  17.12
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'bbmat') , ['A_mem_footprint']] =  20.42
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'conf5_4-8x8-15') , ['A_mem_footprint']] =  22.13
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'mc2depi') , ['A_mem_footprint']] =  26.04
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'rma10') , ['A_mem_footprint']] =  27.35
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'cop20k_A') , ['A_mem_footprint']] =  30.5
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'webbase-1M') , ['A_mem_footprint']] =  39.35
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'cant') , ['A_mem_footprint']] =  46.1
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'pdb1HYS') , ['A_mem_footprint']] =  49.86
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'TSOPF_RS_b300_c3') , ['A_mem_footprint']] =  50.67
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'Chebyshev4') , ['A_mem_footprint']] =  61.8
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'consph') , ['A_mem_footprint']] =  69.1
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'shipsec1') , ['A_mem_footprint']] =  89.95
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'PR02R') , ['A_mem_footprint']] =  94.29
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'mip1') , ['A_mem_footprint']] =  118.73
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'rail4284') , ['A_mem_footprint']] =  129.15
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'pwtk') , ['A_mem_footprint']] =  133.98
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'crankseg_2') , ['A_mem_footprint']] =  162.16
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'Si41Ge41H72') , ['A_mem_footprint']] =  172.5
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'TSOPF_RS_b2383') , ['A_mem_footprint']] =  185.21
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'in-2004') , ['A_mem_footprint']] =  198.88
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'Ga41As41H72') , ['A_mem_footprint']] =  212.61
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'eu-2005') , ['A_mem_footprint']] =  223.42
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'wikipedia-20051105') , ['A_mem_footprint']] =  232.29
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'ldoor') , ['A_mem_footprint']] =  536.04
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'circuit5M') , ['A_mem_footprint']] =  702.4
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'bone010') , ['A_mem_footprint']] =  823.92
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'cage15') , ['A_mem_footprint']] =  1154.91

    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'scircuit') , ['nz']] = 11238432
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'mac_econ_fwd500') , ['nz']] = 8128608
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'raefsky3') , ['nz']] = 1984768
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'bbmat') , ['nz']] = 3699584
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'conf5_4-8x8-15') , ['nz']] = 5844224
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'mc2depi') , ['nz']] = 19709728
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'rma10') , ['nz']] = 4365504
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'cop20k_A') , ['nz']] = 20706112
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'webbase-1M') , ['nz']] = 67805984
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'cant') , ['nz']] = 7189408
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'pdb1HYS') , ['nz']] = 6252896
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'TSOPF_RS_b300_c3') , ['nz']] = 7218400
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'Chebyshev4') , ['nz']] = 10960160
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'consph') , ['nz']] = 11880576
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'shipsec1') , ['nz']] = 13513344
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'PR02R') , ['nz']] = 19352800
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'mip1') , ['nz']] = 18037152
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'rail4284') , ['nz']] = 15567808
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'pwtk') , ['nz']] = 25375072
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'crankseg_2') , ['nz']] = 23444224
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'Si41Ge41H72') , ['nz']] = 65859136
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'TSOPF_RS_b2383') , ['nz']] = 18584608
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'in-2004') , ['nz']] = 58338848
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'Ga41As41H72') , ['nz']] = 100076224
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'eu-2005') , ['nz']] = 93214656
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'wikipedia-20051105') , ['nz']] = 420560672
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'ldoor') , ['nz']] = 115256640
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'circuit5M') , ['nz']] = 756326272
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'bone010') , ['nz']] = 140701056
    group_val_system_best.loc[(group_val_system_best['System'] == 'Alveo-U280') &(group_val_system_best['mtx_name'] == 'cage15') , ['nz']] = 1415024096

    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'scircuit') , ['A_mem_footprint']] =  11.63
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'mac_econ_fwd500') , ['A_mem_footprint']] =  15.36
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'raefsky3') , ['A_mem_footprint']] =  17.12
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'bbmat') , ['A_mem_footprint']] =  20.42
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'conf5_4-8x8-15') , ['A_mem_footprint']] =  22.13
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'mc2depi') , ['A_mem_footprint']] =  26.04
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'rma10') , ['A_mem_footprint']] =  27.35
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'cop20k_A') , ['A_mem_footprint']] =  30.5
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'webbase-1M') , ['A_mem_footprint']] =  39.35
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'cant') , ['A_mem_footprint']] =  46.1
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'pdb1HYS') , ['A_mem_footprint']] =  49.86
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'TSOPF_RS_b300_c3') , ['A_mem_footprint']] =  50.67
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'Chebyshev4') , ['A_mem_footprint']] =  61.8
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'consph') , ['A_mem_footprint']] =  69.1
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'shipsec1') , ['A_mem_footprint']] =  89.95
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'PR02R') , ['A_mem_footprint']] =  94.29
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'mip1') , ['A_mem_footprint']] =  118.73
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'rail4284') , ['A_mem_footprint']] =  129.15
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'pwtk') , ['A_mem_footprint']] =  133.98
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'crankseg_2') , ['A_mem_footprint']] =  162.16
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'Si41Ge41H72') , ['A_mem_footprint']] =  172.5
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'TSOPF_RS_b2383') , ['A_mem_footprint']] =  185.21
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'in-2004') , ['A_mem_footprint']] =  198.88
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'Ga41As41H72') , ['A_mem_footprint']] =  212.61
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'eu-2005') , ['A_mem_footprint']] =  223.42
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'wikipedia-20051105') , ['A_mem_footprint']] =  232.29
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'ldoor') , ['A_mem_footprint']] =  536.04
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'circuit5M') , ['A_mem_footprint']] =  702.4
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'bone010') , ['A_mem_footprint']] =  823.92
    group_val_system_best.loc[(group_val_system_best['System'] == 'Tesla-V100') &(group_val_system_best['mtx_name'] == 'cage15') , ['A_mem_footprint']] =  1154.91

# begin plotting
# perhaps figure size needs refinement, you know better

In [ ]:
    def get_em_feats(pcg, features, dataframe, row, threshold=1):
        low = 1 - pcg*0.01
        high = 1 + pcg*0.01
        first = True
        feature_boolean = True
        for i in range(len(features)):
            feature = features[i]
            feature_boolean_tmp = feature_boolean &\
                                  (dataframe[feature] <= row[feature]*high) &\
                                  (dataframe[feature] >= row[feature]*low)
            
            isol = dataframe[feature_boolean_tmp]
            pcg2 = pcg
            # try to find twins only when skew_coeff value is low
            if((feature == 'skew_coeff' and row[feature] < 5) or (feature == 'avg_num_neighbours') or (feature == 'cross_row_similarity')):
                while(isol.shape[0]<threshold):
                    pcg2 = pcg2+1
                    if((feature == 'avg_num_neighbours' and pcg2>50) or (feature == 'cross_row_similarity' and pcg2>50) or pcg2>1000):
                        break
                    low2 = 1 - pcg2*0.01
                    high2 = 1 + pcg2*0.01
                    feature_boolean_tmp = feature_boolean &\
                                          (dataframe[feature] <= row[feature]*high2) &\
                                          (dataframe[feature] >= row[feature]*low2)
                    isol = dataframe[feature_boolean_tmp]

#             if(pcg2!=pcg):
#                 if(feature=='skew_coeff'):
#                     print('\n',feature, '\t\t', pcg,'->', pcg2, '\t', row['mtx_name'],'\t', row[feature])
#                 else:
#                     print('\n',feature, '\t', pcg,'->', pcg2, '\t', row['mtx_name'],'\t', row[feature])
#                 if(feature !='m'):# and feature !='avg_nz_row'):
#                     print(row['mtx_name'], features[i], row[feature], isol.shape)
#                     print(sorted(set([x for x in isol[feature]])))
#                 if(feature !='m'):
#                     if(i<len(features)-1):
#                         print('\t', features[i+1], sorted(set([np.round(x,2) for x in isol[features[i+1]]])), '\n')

            feature_boolean = feature_boolean_tmp

        similar_synthetic = dataframe[feature_boolean]
        return similar_synthetic

# used to be like this
#         similar_synthetic = group_system_best_part[True \
#     #                                                   & (group_system_best_part['nz'] <= row['nz']*1.3) \
#     #                                                   & (group_system_best_part['nz'] >= row['nz']*0.7) \
#                                                       & (group_system_best_part['m'] <= row['m']*1.3) \
#                                                       & (group_system_best_part['m'] >= row['m']*0.7) \
#                                                       & (group_system_best_part['avg_nz_row'] <= row['avg_nz_row']*1.3) \
#                                                       & (group_system_best_part['avg_nz_row'] >= row['avg_nz_row']*0.7) \
#     #                                                   & (group_system_best_part['std_nz_row'] <= row['std_nz_row']*1.3) \
#     #                                                   & (group_system_best_part['std_nz_row'] >= row['std_nz_row']*0.7) \
#     #                                                   & (group_system_best_part['avg_bandwidth_scaled'] <= row['avg_bandwidth_scaled']*1.3) \
#     #                                                   & (group_system_best_part['avg_bandwidth_scaled'] >= row['avg_bandwidth_scaled']*0.7) \
#                                                       & (group_system_best_part['skew_coeff'] <= row['skew_coeff']*1.3) \
#                                                       & (group_system_best_part['skew_coeff'] >= row['skew_coeff']*0.7) \
#                                                       & (group_system_best_part['avg_num_neighbours'] <= row['avg_num_neighbours']*1.3) \
#                                                       & (group_system_best_part['avg_num_neighbours'] >= row['avg_num_neighbours']*0.7) \
#                                                       & (group_system_best_part['cross_row_similarity'] <= row['cross_row_similarity']*1.3) \
#                                                       & (group_system_best_part['cross_row_similarity'] >= row['cross_row_similarity']*0.7) \
#                                                      ]


# find at least "threshold" twins for each validation matrix
## set threshold = -1 to skip trying 

In [ ]:
    # threshold = -1
    threshold = 10

In [ ]:
font=8
plt.rc('font', family='serif', serif='Times', size=font)
#plt.rc('text', usetex=True)  
plt.rc('xtick', labelsize=font)
plt.rc('ytick', labelsize=font)
plt.rc('axes', labelsize=font)

# width - column as measured in inkscape
width = 3.487
height = width / 1.618

# width - double columnt
width_2col = 7.2
height_2col = width / 1.618

#axs1.get_legend().remove()
#axs1.set_xticklabels([])
#axs1.set_xlabel('')
#axs1.set_ylabel('Relative Normalized Performance')
#axs1.set_xticks([])

# Roofline Stuff

In [ ]:
    def flops_per_byte_spmv_csr(m,n,nnz,dsize,idxdsize):
        return 2.0 * nnz / (dsize*(m+n+nnz) + idxdsize*(m+1 + nnz)) 
        
    group_val_system_best['flops_per_byte'] = flops_per_byte_spmv_csr(group_val_system_best['m'], group_val_system_best['n'], group_val_system_best['nz'], 8, 4)
    group_val_system_best['gflops_cache_roof'] = group_val_system_best['flops_per_byte']*group_val_system_best['cache_bw_gbytes_s']
    group_val_system_best['gflops_mem_roof'] = group_val_system_best['flops_per_byte']*group_val_system_best['mem_bw_gbytes_s']
    print(group_val_system_best)   

# Plot loop Validation

In [ ]:
    ctr = 0
    validation_metadata_systems = []
    #ranges_dev = ["Tesla-P100", "Tesla-V100", "Tesla-A100","Amd-Epyc", "Arm-Neon", "Alveo-U280"] #"TeslaA100",
    # good_sys_names = ["Tesla-P100", "Tesla-V100", "Tesla-A100","Amd-Epyc", "Arm-Neon", "Alveo-U280"] #"TeslaA100",
    ranges_dev = ["Tesla-P100", "Tesla-V100", "Amd-Epyc", "Arm-Neon", "Alveo-U280"] #"TeslaA100",
    good_sys_names = ["Tesla-P100", "Tesla-V100", "Amd-Epyc", "Arm-Neon", "Alveo-U280"] #"TeslaA100",
    fig, axs = plt.subplots(len(ranges_dev))
    fig.subplots_adjust(left=.12, bottom=.17*(2/5)*(4/3.5), right=.99, top=.94)
    for system in ranges_dev:
        plot_x_axis = 'mtx_name'
        plot_y_axis = 'gflops'
        plot_y_axis_memroof = 'gflops_mem_roof'
        plot_y_axis_cacheroof = 'gflops_cache_roof'
        # when you want to plot overall best format
        impl = ''

        # when you want to plot specific format
        # impl = 'cuSPARSE_coo11'
        # impl = 'cuSPARSE_csr11'
        # impl = 'cuSPARSE_hyb9-2'
        # impl = 'CSR5_9'

        select_criterion = ( group_val_system_best['System'] == system )
        select_str = 'Validation-'+system+'\n'+'rows ±30%, avg nonzeros per row ±30%, skew_coeff ±30%,'+'\n' + 'avg_num_neighbours ±30%, cross_row_similarity ±30%'
        if(impl!=''):
            select_criterion =  select_criterion & (group_val_system_best['implementation'] == impl)
            select_str = impl+"_"+ select_str

        groupvalreps_select_best = group_val_system_best[select_criterion]

        groupvalreps_select_sorted = groupvalreps_select_best.sort_values('A_mem_footprint')
        #groupvalreps_select_sorted = groupvalreps_select_best.sort_values('gflops')

        group_system_best_part = group_system_best[group_system_best['System'] == system]

        friends = True
        if(friends != ''):
            group_system_best_part = group_system_best_part[group_system_best_part['friends']==friends]
            if(impl!=''):
                group_system_best_part = group_system_best_part[group_system_best_part['implementation']==impl]

        df_empty = True
        # exclude_matrices = [
        #     "wikipedia-20051105","scircuit","mac_econ_fwd500","rail4284","circuit5M","conf5_4-8x8-15","rajat31","in-2004","eu-2005","TSOPF_RS_b300_c3","mip1","PR02R","Ga41As41H72","shipsec1","Si41Ge41H72","crankseg_2","TSOPF_RS_b2383",
        #     "mc2depi","raefsky3","cop20k_A","bbmat","Chebyshev4","rma10","cage15","cant","pdb1HYS","consph","ldoor","pwtk","bone010" #,"webbase-1M"
        # ]  
        # groupvalreps_select_sorted = groupvalreps_select_sorted[~groupvalreps_select_sorted['mtx_name'].isin(exclude_matrices)]

        validation_headers = ['mtx_name', 'val_gflops', 'synth_mean', 'synth_median', 'synth_min', 'synth_max', 'diff_median', 'diff_mean', 'diff_min', 'diff_max', 'diff_MAPE', 'diff_best_friend_APE']
        validation_metadata = pd.DataFrame(columns=validation_headers)
        
        for index, row in groupvalreps_select_sorted.iterrows():
            pd.options.mode.chained_assignment = None  # default='warn'
            pcg = 30 # +-30%         
            # features to consider too : 'nz', 'std_nz_row', 'avg_bandwidth_scaled'
            features = ['m', 'avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
            similar_synthetic = get_em_feats(pcg, features, group_system_best_part, row, threshold)

            if(impl==''):
                mtx_name_ext = "".join([row['mtx_name']," (",str((row['implementation'])), " - ",str(similar_synthetic.shape[0]), ")"])
            else:
                mtx_name_ext = "".join([row['mtx_name']," (",str(round(row['A_mem_footprint'],1)), " - ",str(similar_synthetic.shape[0]), ")"])
            #similar_synthetic['mtx_name'] = mtx_name_ext Adds Implementation and num of friends in name
            similar_synthetic['mtx_name'] = row['mtx_name']
            
            # box_stats are only useful for "min" and "max".  (https://www.adamsmith.haus/python/docs/matplotlib.cbook.boxplot_stats)
            # for "mean" and "median", same values as simple queries to dataframe are returned
            box_stats = boxplot_stats(similar_synthetic['gflops'])[0]
            synth_mean = box_stats['mean'] # similar_synthetic['gflops'].mean()
            synth_median = box_stats['med'] # similar_synthetic['gflops'].median()
            # these two are the "whiskers" of the boxplot, excluding outliers from min and max
            synth_min = box_stats['whislo']
            synth_max = box_stats['whishi']
                
            if(row['gflops']>0):
                diff_mean = (synth_mean - row['gflops'])/row['gflops'] *100
                diff_median = (synth_median - row['gflops'])/row['gflops'] *100
                diff_min = (synth_min - row['gflops'])/row['gflops'] *100
                diff_max = (synth_max - row['gflops'])/row['gflops'] *100
                diff_MAPE = abs(synth_mean - row['gflops'])/row['gflops'] *100
                diff_best_friend_APE = 10000000
                for index, friend_matrix in similar_synthetic.iterrows():
                    temp_diff_best_friend_APE = abs(friend_matrix['gflops'] - row['gflops'])/row['gflops'] *100
                    if diff_best_friend_APE > temp_diff_best_friend_APE:
                        diff_best_friend_APE = temp_diff_best_friend_APE
            else:
                diff_mean, diff_median, diff_min, diff_max, diff_MAPE, diff_best_friend_APE = 0,0,0,0,0,0

            #print(row['mtx_name'], '\t', row['gflops'])
            #print(synth_mean, '\t', synth_median, '\t', synth_min, '\t', synth_max)
            #print(diff_mean, '\t', diff_median, '\t', diff_min, '\t', diff_max)
            #print(diff_MAPE, '\t', diff_best_friend_APE)

            val_md =  [row['mtx_name'], row['gflops'], 
                       similar_synthetic['gflops'].mean(), similar_synthetic['gflops'].median(), similar_synthetic['gflops'].min(), similar_synthetic['gflops'].max(), 
                       diff_mean, diff_median, diff_min, diff_max, diff_MAPE, diff_best_friend_APE]
            
            if similar_synthetic.empty:
                similar_synthetic = pd.DataFrame([[mtx_name_ext] + [0]*(len(group_system_best_part.columns)-1)], columns = group_system_best_part.columns)
            if(row['gflops']==-1):
                similar_synthetic = pd.DataFrame([[mtx_name_ext] + [0]*(len(group_system_best_part.columns)-1)], columns = group_system_best_part.columns)
            if df_empty:
                synthetic_neighbors = similar_synthetic
                validation_metadata.loc[0] = val_md
                df_empty = False
            else:
                synthetic_neighbors = pd.concat([synthetic_neighbors, similar_synthetic])
                validation_metadata.loc[len(validation_metadata)] = val_md
            
        select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')

        sns_plot = sns.boxplot(data=synthetic_neighbors, x='mtx_name', y=plot_y_axis, ax=axs[ctr], fliersize = 0.5, linewidth = 0.5, color = cp6[ctr],
                              # showmeans=True,meanprops={"marker":"s","markerfacecolor":"green", "markeredgecolor":"blue","markersize":"2"}
                              ) # linewidth is 0
        #print(synthetic_neighbors)
        sns_plot = sns.scatterplot(data=groupvalreps_select_sorted, x=plot_x_axis, y=plot_y_axis, ax=axs[ctr], s = 10, color = 'k')
        #print(groupvalreps_select_sorted)
        if system == "Amd-Epyc" or system == "Arm-Neon":
            sns_plot_llc = sns.lineplot(data=groupvalreps_select_sorted, x=plot_x_axis, y=plot_y_axis_cacheroof, ax=axs[ctr], marker='X', markersize = 3, linestyle = '--', linewidth = 0.5, color = 'gray', markeredgewidth=0)
        sns_plot_mem = sns.lineplot(data=groupvalreps_select_sorted, x=plot_x_axis, y=plot_y_axis_memroof, ax=axs[ctr], marker='^', markersize = 3, linestyle = '--', linewidth = 0.5, color = 'gray', markeredgewidth=0)
        
        #axs.set_title(system)# axs.set_title('Dataset %s' %(select_str))
        #axs.get_legend().remove()
  
        axs[ctr].tick_params(axis='both', which='major', labelsize= font-3)
        axs[ctr].tick_params(axis='both', which='minor', labelsize= font-3)
        axs[ctr].set_xticklabels([])
        if ctr == len(ranges_dev)//2:
            axs[ctr].set_ylabel('GFLOPs', fontsize = font)
        else:
            axs[ctr].set_ylabel('')
        if ctr == len(ranges_dev) - 1:
            axs[ctr].set_xlabel('Matrix Id (Table II)', fontsize = font)
        else:
            axs[ctr].set_xlabel('')
        if ctr == len(ranges_dev) - 1:
            axs[ctr].set_xticklabels(xtick_labels, fontsize = font-4)
        else:
            xtick_labels = range(1,len(set(synthetic_neighbors['mtx_name']))+1)
            axs[ctr].set_xticks([])
            axs[ctr].set_xticklabels([])
        axs[ctr].text( 0.05, 0.8, good_sys_names[ctr], fontsize=font, transform = axs[ctr].transAxes, )
        if ctr == len(ranges_dev) - 1:
            axs[ctr].set_ylim([0, axs[ctr].get_ylim()[1]*1.3])
        else:
            axs[ctr].set_ylim([axs[ctr].get_ylim()[0], axs[ctr].get_ylim()[1]*1.2])
        validation_metadata_systems.append(validation_metadata)
        ctr+=1
        #plt.xticks(rotation=90)
    plt.subplots_adjust(wspace=0, hspace=0)
    
    # Legendary stuff
    import matplotlib.lines as mlines
    mem_roof = mlines.Line2D([], [], color='gray', marker='^',
                          markersize=3, linestyle = '--', linewidth = 0.5, label='Roofline Memory')
    llc_roof = mlines.Line2D([], [], color='gray', marker='X',
                      markersize=3, linestyle = '--', linewidth = 0.5, label='Roofline LLC')
    axs[0].legend(handles=[mem_roof, llc_roof], bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
    
    fig.set_size_inches(width, height/(2/5)*3.5/4)
    fig.savefig('./Paper_plots/Validation_merged-all_x-%s_y-%s.pdf' % (plot_x_axis, plot_y_axis))
    plt.close()


In [ ]:
#     ctr = 0
#     validation_metadata_systems = []
#     ranges_dev = ["TeslaV100", "TeslaP100", "HawkAmdRome", "Arm", "AlveoU280"]
#     for system in ranges_dev:
#         plot_x_axis = 'mtx_name'
#         plot_y_axis = 'gflops'

#         # when you want to plot overall best format
#         impl = ''

#         # when you want to plot specific format
#         # impl = 'cuSPARSE_coo11'
#         # impl = 'cuSPARSE_csr11'
#         # impl = 'cuSPARSE_hyb9-2'
#         # impl = 'CSR5_9'

#         select_criterion = ( group_val_system_best['System'] == system )
#         select_str = 'Validation-'+system+'\n'+'rows ±30%, avg nonzeros per row ±30%, skew_coeff ±30%,'+'\n' + 'avg_num_neighbours ±30%, cross_row_similarity ±30%'
#         if(impl!=''):
#             select_criterion =  select_criterion & (group_val_system_best['implementation'] == impl)
#             select_str = impl+"_"+ select_str

#         groupvalreps_select_best = group_val_system_best[select_criterion]

#         groupvalreps_select_sorted = groupvalreps_select_best.sort_values('A_mem_footprint')
#         #groupvalreps_select_sorted = groupvalreps_select_best.sort_values('gflops')

#         group_system_best_part = group_system_best[group_system_best['System'] == system]

#         friends = True
#         if(friends != ''):
#             group_system_best_part = group_system_best_part[group_system_best_part['friends']==friends]
#             if(impl!=''):
#                 group_system_best_part = group_system_best_part[group_system_best_part['implementation']==impl]

#         df_empty = True
#         # exclude_matrices = [
#         #     "wikipedia-20051105","scircuit","mac_econ_fwd500","rail4284","circuit5M","conf5_4-8x8-15","rajat31","in-2004","eu-2005","TSOPF_RS_b300_c3","mip1","PR02R","Ga41As41H72","shipsec1","Si41Ge41H72","crankseg_2","TSOPF_RS_b2383",
#         #     "mc2depi","raefsky3","cop20k_A","bbmat","Chebyshev4","rma10","cage15","cant","pdb1HYS","consph","ldoor","pwtk","bone010" #,"webbase-1M"
#         # ]  
#         # groupvalreps_select_sorted = groupvalreps_select_sorted[~groupvalreps_select_sorted['mtx_name'].isin(exclude_matrices)]

#         validation_headers = ['mtx_name', 'val_gflops', 'synth_mean', 'synth_median', 'synth_min', 'synth_max', 'diff_median', 'diff_mean', 'diff_min', 'diff_max']
#         validation_metadata = pd.DataFrame(columns=validation_headers)
        
#         for index, row in groupvalreps_select_sorted.iterrows():
#             pd.options.mode.chained_assignment = None  # default='warn'
#             pcg = 30 # +-30%         
#             # features to consider too : 'nz', 'std_nz_row', 'avg_bandwidth_scaled'
#             features = ['m', 'avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
#             similar_synthetic = get_em_feats(pcg, features, group_system_best_part, row, threshold)

#             if(impl==''):
#                 mtx_name_ext = "".join([row['mtx_name']," (",str((row['implementation'])), " - ",str(similar_synthetic.shape[0]), ")"])
#             else:
#                 mtx_name_ext = "".join([row['mtx_name']," (",str(round(row['A_mem_footprint'],1)), " - ",str(similar_synthetic.shape[0]), ")"])
#             #similar_synthetic['mtx_name'] = mtx_name_ext Adds Implementation and num of friends in name
#             similar_synthetic['mtx_name'] = row['mtx_name']
            
#             # box_stats are only useful for "min" and "max".  (https://www.adamsmith.haus/python/docs/matplotlib.cbook.boxplot_stats)
#             # for "mean" and "median", same values as simple queries to dataframe are returned
#             box_stats = boxplot_stats(similar_synthetic['gflops'])[0]
#             synth_mean = box_stats['mean'] # similar_synthetic['gflops'].mean()
#             synth_median = box_stats['med'] # similar_synthetic['gflops'].median()
#             # these two are the "whiskers" of the boxplot, excluding outliers from min and max
#             synth_min = box_stats['whislo']
#             synth_max = box_stats['whishi']

#             if(row['gflops']>0):
#                 diff_mean = (synth_mean - row['gflops'])/row['gflops'] *100
#                 diff_median = (synth_median - row['gflops'])/row['gflops'] *100
#                 diff_min = (synth_min - row['gflops'])/row['gflops'] *100
#                 diff_max = (synth_max - row['gflops'])/row['gflops'] *100
#             else:
#                 diff_mean, diff_median, diff_min, diff_max = 0,0,0,0

#             # print(row['mtx_name'], '\t', row['gflops'])
#             # print(synth_mean, '\t', synth_median, '\t', synth_min, '\t', synth_max)
#             # print(diff_mean, '\t', diff_median, '\t', diff_min, '\t', diff_max)

#             val_md =  [row['mtx_name'], row['gflops'], 
#                        similar_synthetic['gflops'].mean(), similar_synthetic['gflops'].median(), similar_synthetic['gflops'].min(), similar_synthetic['gflops'].max(), 
#                        diff_mean, diff_median, diff_min, diff_max]
            
#             if similar_synthetic.empty:
#                 similar_synthetic = pd.DataFrame([[mtx_name_ext] + [0]*(len(group_system_best_part.columns)-1)], columns = group_system_best_part.columns)
#             if df_empty:
#                 synthetic_neighbors = similar_synthetic
#                 validation_metadata.loc[0] = val_md
#                 df_empty = False
#             else:
#                 synthetic_neighbors = pd.concat([synthetic_neighbors, similar_synthetic])
#                 validation_metadata.loc[len(validation_metadata)] = val_md
            
#         select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
#         fig, axs = plt.subplots()
#         fig.subplots_adjust(left=.14, bottom=.25, right=.99, top=.98)

#         sns_plot = sns.boxplot(data=synthetic_neighbors, x='mtx_name', y=plot_y_axis, ax=axs, fliersize = 0.5, linewidth = 0.7, color = cp6[ctr],
#                               # showmeans=True,meanprops={"marker":"s","markerfacecolor":"green", "markeredgecolor":"blue","markersize":"2"}
#                               ) # linewidth is 0
#         sns_plot = sns.scatterplot(data=groupvalreps_select_sorted, x=plot_x_axis, y=plot_y_axis, ax=axs, s = 10, color = 'k')
#         #axs.set_title(system)# axs.set_title('Dataset %s' %(select_str))
#         #axs.get_legend().remove()
        
#         axs.set_ylabel('Performance(Gflops/s)', fontsize = font-1)
#         axs.set_xticklabels(range(0,len(set(synthetic_neighbors['mtx_name']))), fontsize = font-2)
#         axs.set_xlabel('Mtx Id', fontsize = font-1)
#         plt.xticks(rotation=90)
#         fig.set_size_inches(width, height/(3/2))
#         #fig.savefig('./Paper_plots/%s_x-%s_y-%s.pdf' % (select_str_id, plot_x_axis, plot_y_axis))
#         fig.savefig('./Paper_plots/Validation_system-%s_x-%s_y-%s.pdf' % (system, plot_x_axis, plot_y_axis))
#         plt.close()
#         ctr+=1
        
#         validation_metadata_systems.append(validation_metadata)
#         # break

# "validation_metadata_systems"  is a list of metadata for each system containing a dataframe with following columns
# ['mtx_name', 'val_gflops', 'synth_mean', 'synth_median', 'synth_min', 'synth_max', 'diff_median', 'diff_mean', 'diff_min', 'diff_max']

In [ ]:
def keep_good_stuff(which_column, validation_metadata_systems):
    # tmp_validation_metadata_df = pd.DataFrame(columns=validation_headers)
    # testing = 'diff_median_'
    testing = ''
    for i in range(len(ranges_dev)):    
        validation_metadata_systems[i].rename(columns={which_column:ranges_dev[i]}, inplace=True)
        if(i==0):
            tmp_validation_metadata_df = validation_metadata_systems[i]
        else:
            # validation_metadata_df = pd.concat([validation_metadata_df,validation_metadata_systems[i]], axis=1)
            # https://datacarpentry.org/python-socialsci/11-joins/index.html
            #tmp_validation_metadata_df = pd.merge(tmp_validation_metadata_df,validation_metadata_systems_backup[i], how='left', on = 'mtx_name')
            #/tmp/ipykernel_24490/4187708563.py:13: FutureWarning: Passing 'suffixes' which cause duplicate columns {'val_gflops_x', 'diff_max_x', 'synth_mean_x', 'diff_median_x', 'synth_max_x', 'diff_mean_x', 'diff_min_x', 'synth_min_x', 'synth_median_x'} in the result is deprecated and will raise a MergeError in a future version.
            tmp_validation_metadata_df = pd.merge(tmp_validation_metadata_df,validation_metadata_systems[i], how='left', on = 'mtx_name') 
    validation_metadata_df = tmp_validation_metadata_df[['mtx_name']+ [i for i in ranges_dev]]
    return validation_metadata_df


# keep only "diff_median" columns
# whichever column is selected, will be the only one printed afterwards
# which_column = 'diff_MAPE'
# validation_metadata_df = keep_good_stuff(which_column, validation_metadata_systems)
# print(validation_metadata_df.columns)
# print(validation_metadata_df)
# validation_metadata_df.to_csv("validation_vs_friends_MAPE.csv")

which_column = 'diff_MAPE'
validation_metadata_df = keep_good_stuff(which_column, validation_metadata_systems)
print(validation_metadata_df.columns)
print(validation_metadata_df)
validation_metadata_df.to_csv("validation_vs_friends_MAPE.csv")

---
# produce separate plots per testbed, not needed anymore

In [ ]:
    # ctr = 0
    # for system in ['Tesla-V100', 'Tesla-P100', 'Amd-Epyc', 'Arm-Neon','Alveo-U280']:
    #     plot_x_axis = 'mtx_name'
    #     plot_y_axis = 'gflops'

    #     # when you want to plot overall best format
    #     impl = ''

    #     # when you want to plot specific format
    # #     impl = 'cuSPARSE_coo11'
    # #     impl = 'cuSPARSE_csr11'
    # #     impl = 'cuSPARSE_hyb9-2'
    # #     impl = 'CSR5_9'

    #     select_criterion = ( group_val_system_best['System'] == system )
    #     select_str = 'Validation-'+system+'\n'+'rows ±30%, avg nonzeros per row ±30%, skew_coeff ±30%,'+'\n' + 'avg_num_neighbours ±30%, cross_row_similarity ±30%'
    #     if(impl!=''):
    #         select_criterion =  select_criterion & (group_val_system_best['implementation'] == impl)
    #         select_str = impl+"_"+ select_str

    #     groupvalreps_select_best = group_val_system_best[select_criterion]

    #     groupvalreps_select_sorted = groupvalreps_select_best.sort_values('A_mem_footprint')
    #     #groupvalreps_select_sorted = groupvalreps_select_best.sort_values('gflops')

    #     group_system_best_part = group_system_best[group_system_best['System'] == system]

    #     friends = True
    #     if(friends != ''):
    #         group_system_best_part = group_system_best_part[group_system_best_part['friends']==friends]
    #         if(impl!=''):
    #             group_system_best_part = group_system_best_part[group_system_best_part['implementation']==impl]

    #     df_empty = True

    # #     exclude_matrices = [
    # #         "wikipedia-20051105","scircuit","mac_econ_fwd500","rail4284","circuit5M","conf5_4-8x8-15","rajat31","in-2004","eu-2005","TSOPF_RS_b300_c3","mip1","PR02R","Ga41As41H72","shipsec1","Si41Ge41H72","crankseg_2","TSOPF_RS_b2383",
    # #         "mc2depi","raefsky3","cop20k_A","bbmat","Chebyshev4","rma10","cage15","cant","pdb1HYS","consph","ldoor","pwtk","bone010" #,"webbase-1M"
    # #     ]  
    # #     groupvalreps_select_sorted = groupvalreps_select_sorted[~groupvalreps_select_sorted['mtx_name'].isin(exclude_matrices)]

    #     for index, row in groupvalreps_select_sorted.iterrows():
    #         pd.options.mode.chained_assignment = None  # default='warn'
    #         pcg = 30 # +-30%         
    #         # features to consider too : 'nz', 'std_nz_row', 'avg_bandwidth_scaled'
    #         features = ['m', 'avg_nz_row', 'skew_coeff', 'avg_num_neighbours', 'cross_row_similarity']
    #         similar_synthetic = get_em_feats(pcg, features, group_system_best_part, row, threshold)

    #         if(impl==''):
    #             mtx_name_ext = "".join([row['mtx_name']," (",str((row['implementation'])), " - ",str(similar_synthetic.shape[0]), ")"])
    #         else:
    #             mtx_name_ext = "".join([row['mtx_name']," (",str(round(row['A_mem_footprint'],1)), " - ",str(similar_synthetic.shape[0]), ")"])
    #         #similar_synthetic['mtx_name'] = mtx_name_ext Adds Implementation and num of friends in name
    #         similar_synthetic['mtx_name'] = row['mtx_name']

    #         if similar_synthetic.empty:
    #             similar_synthetic = pd.DataFrame([[mtx_name_ext] + [0]*(len(group_system_best_part.columns)-1)], columns = group_system_best_part.columns)
    #         if df_empty:
    #             synthetic_neighbors = similar_synthetic
    #             df_empty = False
    #         else:
    #             synthetic_neighbors = pd.concat([synthetic_neighbors, similar_synthetic])

    #     select_str_id = select_str.replace('=', 'eq').replace('<', 'l').replace('>', 'g').replace(', ', '_').replace(',', '_').replace('\n', '_').replace(' ', '-')
    #     fig, axs = plt.subplots()
    #     fig.subplots_adjust(left=.14, bottom=.02, right=.99, top=.96)
    #     sns_plot = sns.boxplot(data=synthetic_neighbors, x='mtx_name', y=plot_y_axis, ax=axs, fliersize = 0.5, linewidth = 0., color = cp6[ctr])
    #     sns_plot = sns.scatterplot(data=groupvalreps_select_sorted, x=plot_x_axis, y=plot_y_axis, ax=axs, s = 10, color = 'k')
    #     #axs.set_title(system)# axs.set_title('Dataset %s' %(select_str))
    #     #axs.get_legend().remove()
    #     axs.set_ylabel('GFLOPs', fontsize = font-1)
    #     axs.set_xticklabels([])
    #     axs.set_xticks([])
    #     axs.set_xlabel('')
    #     plt.xticks(rotation=90)
    #     fig.set_size_inches(width, height/(5/3))
    #     #fig.savefig('./Paper_plots/%s_x-%s_y-%s.pdf' % (select_str_id, plot_x_axis, plot_y_axis))
    #     fig.savefig('./Paper_plots/Validation_cropped_system-%s_x-%s_y-%s.pdf' % (system, plot_x_axis, plot_y_axis))
    #     plt.close()
    #     ctr+=1